In [73]:
import pandas as pd
import altair as alt
from scipy import stats
import numpy as np

```python
        state.now = parseFloat(values[0]);
        state.rpm = parseFloat(values[1]);
        state.speed = parseFloat(values[2]);
        state.tacho = parseFloat(values[3]);
        state.distance = parseFloat(values[4]);
        state.voltage = parseFloat(values[5]);
        state.temp_mos = parseFloat(values[6]);
        state.temp_motor = parseFloat(values[7]);
        state.current_motor = parseFloat(values[8]);
        state.current_batt = parseFloat(values[9]);
        state.net_energy = parseFloat(values[23]);
        state.efficiency = parseFloat(values[24]);
        state.avg_speed = parseFloat(values[25]);
        state.elapsed = parseFloat(values[26]);
        state.time_remaining = parseFloat(values[27]);
        state.fault_code = parseFloat(values[20])
```

In [74]:
def urban_theme():
    markColor = "#1696d2"
    axisColor = "#000000"
    backgroundColor = "#FFFFFF"
    font = "Lato"
    labelFont = "Lato"
    sourceFont = "Lato"
    gridColor = "#DEDDDD"
    main_palette = ["#1696d2",
                    "#d2d2d2",
                    "#000000",
                    "#fdbf11",
                    "#ec008b",
                    "#55b748",
                    "#5c5859",
                    "#db2b27",
                    ]
    sequential_palette = ["#cfe8f3",
                          "#a2d4ec",
                          "#73bfe2",
                          "#46abdb",
                          "#1696d2",
                          "#12719e",
                          ]
    return {
        "width": 685,
        "height": 380,
        #        "autosize": "fit",
        "config": {
            "title": {
                "anchor": "start",
                "fontSize": 18,
                "font": font,
                "fontColor": "#000000"
            },
            "axisX": {
                "domain": True,
                "domainColor": axisColor,
                "domainWidth": 1,
                "grid": False,
                "labelFontSize": 12,
                "labelFont": labelFont,
                "labelAngle": 0,
                "tickColor": axisColor,
                "tickSize": 5,
                "titleFontSize": 12,
                "titlePadding": 10,
                "titleFont": font,
                "title": "",
            },
            "axisY": {
                "domain": False,
                "grid": True,
                "gridColor": gridColor,
                "gridWidth": 1,
                "labelFontSize": 12,
                "labelFont": labelFont,
                "labelPadding": 8,
                "ticks": False,
                "titleFontSize": 12,
                "titlePadding": 10,
                "titleFont": font,
                "titleAngle": 0,
                "titleY": -10,
                "titleX": 18,
            },
            "background": backgroundColor,
            "legend": {
                "labelFontSize": 12,
                "labelFont": labelFont,
                "symbolSize": 100,
                "symbolType": "square",
                "titleFontSize": 12,
                "titlePadding": 10,
                "titleFont": font,
                "title": "",
                "orient": "top-left",
                "offset": 0,
            },
            "view": {
                "stroke": "transparent",
            },
            "range": {
                "category": main_palette,
                "diverging": sequential_palette,
            },
            "area": {
                "fill": markColor,
            },
            "line": {
                "color": markColor,
                "stroke": markColor,
                "strokewidth": 5,
            },
            "trail": {
                "color": markColor,
                "stroke": markColor,
                "strokeWidth": 0,
                "size": 1,
            },
            "path": {
                "stroke": markColor,
                "strokeWidth": 0.5,
            },
            "point": {
                "filled": True,
            },
            "text": {
                "font": sourceFont,
                "color": markColor,
                "fontSize": 11,
                "align": "right",
                "fontWeight": 400,
                "size": 11,
            },
            "bar": {
                "size": 40,
                "binSpacing": 1,
                "continuousBandSize": 30,
                "discreteBandSize": 30,
                "fill": markColor,
                "stroke": False,
            },
        },
    }


alt.themes.register("my_custom_theme", urban_theme)
alt.themes.enable("my_custom_theme")

ThemeRegistry.enable('my_custom_theme')

In [75]:
df = pd.read_csv("drive-download-20240226T231824Z-001/Run1.csv", header=None)
rename_dict = {
    0: "now",
    1: "rpm",
    2: "speed",
    3: "tacho",
    4: "distance",
    8: "current_motor",
    9: "current_bat"
}
df = df.rename(columns=rename_dict)

In [76]:
def mask_df(df):
    start = df["now"][0]
    df["time"] = df["now"] - start
    run_df = df[df["speed"] > 0]
    start_time = run_df["time"].iloc[0]
    end_time = run_df["time"].iloc[-1]
    masked = df[(df["time"] > start_time) & (df["time"] < end_time)]
    return masked

In [82]:
masked = mask_df(df)


#rolling average over speeds
def get_rolling(attribute, masked):
    copy = masked.copy()
    copy = copy[(copy[attribute] < 50)]
    rolling = copy[attribute].rolling(window=3).mean()
    rolling = rolling[rolling.notna()]
    return rolling

def graph_attribute(attribute, masked, rolling_window = 3):
    copy = masked.copy()
    copy = copy[copy[attribute] < 50]
    copy["rolling"] = copy[attribute].rolling(window=rolling_window).mean()
    copy = copy[copy["rolling"].notna()]
    copy = copy[copy[attribute] > 1]
    chart = alt.Chart(copy[["time", "rolling"]]).mark_line().encode(
        x=alt.X("time", title="Time (s)"),
        y=alt.Y("rolling", title="Velocity (m/s)"))
    display(chart)


masked = masked[(masked["time"] > 150) & (masked["time"] < 200)]
speed_rolling = get_rolling("speed", masked)
motor_rolling = get_rolling("current_motor", masked)
masked["speed_rolling"] = speed_rolling
masked["motor_rolling"] = motor_rolling
display(masked)
base = alt.Chart(masked[[ "time", "speed_rolling", "motor_rolling" ]]).encode(
    alt.X("time", title="time (s)")
)
line1 = base.mark_line(stroke="#04a5e5").encode(
    alt.Y("speed_rolling", axis=(alt.Axis(title = "Velocity (m/s)", titleColor = "#04a5e5"))) 
)

line2 = base.mark_line(stroke="#fe640b").encode(
    alt.Y("motor_rolling", scale = alt.Scale(domain=[0, 20]), axis=(alt.Axis(title = "Motor Current (Amps)", titleColor = "#fe640b")))
)

alt.layer(line1, line2).resolve_scale(
    y="independent"
)


,now,rpm,speed,tacho,distance,5,6,7,current_motor,current_bat,...,19,20,21,22,23,24,25,time,speed_rolling,motor_rolling
338,1647558711,140.39,10.86,13,0.02,52.3,43.7,-99.4,15.68,8.10,...,1711,0,0.0,0,0.36,55.25,1.44,151,NaN,NaN
339,1647558711,140.39,10.86,13,0.02,53.1,43.7,-88.7,0.32,0.12,...,1795,0,0.0,0,0.36,55.11,1.44,151,NaN,NaN
340,1647558711,140.39,10.86,13,0.02,53.2,43.8,-87.6,0.10,0.00,...,1810,0,0.0,0,0.36,55.11,1.44,151,10.860000,5.366667
341,1647558711,146.26,11.31,14,0.02,53.1,43.9,-88.4,0.22,0.00,...,1810,0,0.0,0,0.36,55.11,1.44,151,11.010000,0.213333
342,1647558712,146.26,11.31,14,0.02,53.2,43.9,-87.2,0.22,0.00,...,1810,0,0.0,0,0.36,55.11,1.41,152,11.160000,0.180000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,1647558758,224.70,17.38,172,0.22,53.0,47.2,-88.4,0.05,0.00,...,11966,0,0.0,0,1.75,125.72,8.16,198,17.426667,0.056667
546,1647558759,223.24,17.27,173,0.22,53.1,47.2,-87.4,0.06,0.00,...,11966,0,0.0,0,1.75,125.72,8.08,199,17.343333,0.056667
547,1647558759,219.97,17.01,175,0.23,53.1,47.2,-88.1,0.06,0.00,...,11966,0,0.0,0,1.75,131.44,8.45,199,17.220000,0.056667
548,1647558759,219.97,17.01,175,0.23,53.0,47.1,-87.0,0.05,0.00,...,11966,0,0.0,0,1.75,131.44,8.45,199,17.096667,0.056667


alt.LayerChart(...)

In [83]:
graph_attribute("speed", masked, 5)

alt.Chart(...)